In [1]:
# from CBAStats.Player import *
# from CBAStats.Team import *
# from CBAStats.Player import stats_output
import datetime
import Scraper
import pandas as pd
from pathlib import Path
from sqlalchemy import create_engine
import pymysql

In [2]:
print('最近一次运行时间：',datetime.datetime.now())

最近一次运行时间： 2020-08-04 01:45:02.306244


# 更新数据

In [3]:
# 定义文件路径，如果文件已存在则会覆盖旧文件
schedule_output = f'../StatsData/GamesSchedulePage_{datetime.date.today()}.csv'
game_stats_output = f'../StatsData/All_Games_Stats_{datetime.date.today()}.csv'
teams_stats_output = f'../StatsData/TeamsAdvancedStats_{datetime.date.today()}.csv'
players_stats_output = f'../StatsData/PlayersAdvancedStats_{datetime.date.today()}.csv'

for output in [schedule_output,game_stats_output,teams_stats_output,players_stats_output]:
    path = Path(output)
    print(f'Checking {output}.')
    # check if folder is reachable
    assert (path.parent.exists()),f"Folder '{path.parent}' does not exist."
    # check if file already exists
    if path.exists():
        print(f'\tFile {path} already exists! It will be overwritten.')
    print('\tPass!')

Checking ../StatsData/GamesSchedulePage_2020-08-04.csv.
	Pass!
Checking ../StatsData/All_Games_Stats_2020-08-04.csv.
	Pass!
Checking ../StatsData/TeamsAdvancedStats_2020-08-04.csv.
	Pass!
Checking ../StatsData/PlayersAdvancedStats_2020-08-04.csv.
	Pass!


In [6]:
# only supports csv file for now
Scraper.scrape_sina_schedule(output_path=schedule_output)
Scraper.scrape_game_details(input_file=schedule_output,
                            output_file=game_stats_output)

开始爬取所有赛程、数据统计链接.
所有赛程、数据统计链接将会存在 ../StatsData/GamesSchedulePage_2020-08-04.csv.
所有赛程、数据统计链接已保存在 ../StatsData/GamesSchedulePage_2020-08-04.csv.
从../StatsData/GamesSchedulePage_2020-08-04.csv读取所有赛程、数据统计链接
读取完毕
抓取比赛  2019202000001   广东   辽宁   2020-08-04 02:10:02.401171
抓取比赛  2019202000002   江苏   浙江   2020-08-04 02:10:05.632555
抓取比赛  2019202000003   四川   新疆   2020-08-04 02:10:09.409176
抓取比赛  2019202000004   山东   八一   2020-08-04 02:10:11.895639
抓取比赛  2019202000005   北控   深圳   2020-08-04 02:10:14.362520
抓取比赛  2019202000006   青岛   吉林   2020-08-04 02:10:15.986781
抓取比赛  2019202000007   广厦   福建   2020-08-04 02:10:18.371574
抓取比赛  2019202000008   上海   同曦   2020-08-04 02:10:20.369829
抓取比赛  2019202000009   天津   北京   2020-08-04 02:10:22.799079
抓取比赛  2019202000010   广州   山西   2020-08-04 02:10:25.099673
抓取比赛  2019202000011   辽宁   广厦   2020-08-04 02:10:27.905085
抓取比赛  2019202000012   北控   上海   2020-08-04 02:10:29.728833
抓取比赛  2019202000013   同曦   江苏   2020-08-04 02:10:32.094166
抓取比赛  2019202000014   山西 

抓取比赛  2019202000137   广州   辽宁   2020-08-04 02:15:38.787315
抓取比赛  2019202000138   北京   深圳   2020-08-04 02:15:41.919627
抓取比赛  2019202000139   八一   江苏   2020-08-04 02:15:44.417084
抓取比赛  2019202000140   吉林   新疆   2020-08-04 02:15:46.009953
抓取比赛  2019202000141   福建   北控   2020-08-04 02:15:47.662459
抓取比赛  2019202000142   吉林   上海   2020-08-04 02:15:52.294125
抓取比赛  2019202000143   北京   广州   2020-08-04 02:15:53.413973
抓取比赛  2019202000144   浙江   八一   2020-08-04 02:15:56.479510
抓取比赛  2019202000145   辽宁   同曦   2020-08-04 02:15:59.407278
抓取比赛  2019202000146   广东   广厦   2020-08-04 02:16:01.189040
抓取比赛  2019202000147   山东   江苏   2020-08-04 02:16:03.794181
抓取比赛  2019202000148   山西   新疆   2020-08-04 02:16:05.078178
抓取比赛  2019202000149   青岛   深圳   2020-08-04 02:16:08.441196
抓取比赛  2019202000150   四川   天津   2020-08-04 02:16:10.345018
抓取比赛  2019202000151   吉林   同曦   2020-08-04 02:16:12.256469
抓取比赛  2019202000152   北控   广州   2020-08-04 02:16:13.453653
抓取比赛  2019202000153   广厦   八一   2020-08-04 02:16:17.0034

抓取比赛  2019202000276   福建   八一   2020-08-04 02:21:43.685381
抓取比赛  2019202000277   吉林   新疆   2020-08-04 02:21:47.758233
抓取比赛  2019202000278   广州   青岛   2020-08-04 02:21:50.887924
抓取比赛  2019202000279   北京   深圳   2020-08-04 02:21:55.153244
抓取比赛  2019202000280   浙江   北控   2020-08-04 02:21:57.691204
抓取比赛  2019202000281   广东   山东   2020-08-04 02:21:58.859683
抓取比赛  2019202000282   同曦   山西   2020-08-04 02:22:00.507941
抓取比赛  2019202000283   吉林   八一   2020-08-04 02:22:03.691635
抓取比赛  2019202000284   辽宁   新疆   2020-08-04 02:22:05.651049
抓取比赛  2019202000285   广州   四川   2020-08-04 02:22:07.273933
抓取比赛  2019202000286   上海   深圳   2020-08-04 02:22:09.132846
抓取比赛  2019202000287   北京   江苏   2020-08-04 02:22:10.055195
抓取比赛  2019202000288   浙江   天津   2020-08-04 02:22:11.824863
抓取比赛  2019202000289   广厦   北控   2020-08-04 02:22:14.892495
抓取比赛  2019202000290   福建   青岛   2020-08-04 02:22:18.102770
抓取比赛  2019202000291   吉林   山西   2020-08-04 02:22:22.840432
抓取比赛  2019202000292   江苏   上海   2020-08-04 02:22:24.0738

抓取比赛  2019202000415   广厦   广东   2020-08-04 02:27:37.336783
抓取比赛  2019202000416   山东   新疆   2020-08-04 02:27:40.936831
抓取比赛  2019202000417   八一   上海   2020-08-04 02:27:45.253437
抓取比赛  2019202000418   青岛   北控   2020-08-04 02:27:49.054294
抓取比赛  2019202000419   同曦   吉林   2020-08-04 02:27:50.491542
抓取比赛  2019202000420   辽宁   福建   2020-08-04 02:27:51.662677
抓取比赛  2019202000421   江苏   四川   2020-08-04 02:27:55.359144
抓取比赛  2019202000422   广州   北京   2020-08-04 02:27:58.475125
抓取比赛  2019202000423   浙江   山西   2020-08-04 02:28:00.608918
抓取比赛  2019202000424   上海   吉林   2020-08-04 02:28:04.330709
抓取比赛  2019202000425   天津   八一   2020-08-04 02:28:05.863097
抓取比赛  2019202000426   北控   辽宁   2020-08-04 02:28:07.632363
抓取比赛  2019202000427   广厦   深圳   2020-08-04 02:28:08.898589
抓取比赛  2019202000428   北京   四川   2020-08-04 02:28:10.458400
抓取比赛  2019202000429   山西   青岛   2020-08-04 02:28:15.399911
抓取比赛  2019202000430   浙江   广东   2020-08-04 02:28:18.545176
抓取比赛  2019202000431   福建   新疆   2020-08-04 02:28:21.2104

# 载入爬取的数据

In [7]:
df = pd.read_csv(game_stats_output,dtype={'号码':'str','Game_ID':'str'})
df

,球员,号码,出场时间,首发,进攻篮板,防守篮板,助攻,犯规,抢断,失误,...,得分,Game_ID,球队,对手,2分中,2分投,3分中,3分投,罚球中,罚球投
0,布鲁克斯,6,36,1,0,3,9,3,2,1,...,27,2019202000001,广东,辽宁,8,10,2,10,5,5
1,任骏飞,20,30,1,1,1,1,3,3,0,...,19,2019202000001,广东,辽宁,5,8,3,6,0,0
2,易建联,9,35,1,1,11,2,2,1,1,...,16,2019202000001,广东,辽宁,7,8,0,0,2,4
3,赵睿,10,31,1,2,5,4,3,2,0,...,10,2019202000001,广东,辽宁,5,9,0,3,0,1
4,胡明轩,3,13,1,0,1,0,1,0,1,...,3,2019202000001,广东,辽宁,0,1,1,3,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16129,孙岩松,9,0,0,0,0,0,0,0,0,...,0,2019202000469,浙江,辽宁,0,0,0,0,0,0
16130,王仔路,23,1,0,0,0,0,0,0,0,...,0,2019202000469,浙江,辽宁,0,0,0,0,0,0
16131,鞠鹏,7,0,0,0,0,0,0,0,0,...,0,2019202000469,浙江,辽宁,0,0,0,0,0,0
16132,张正昕,28,0,0,0,0,0,0,0,0,...,0,2019202000469,浙江,辽宁,0,0,0,0,0,0


# Insert Records

In [8]:
from sqlalchemy import create_engine
import pymysql

user_name = 'master'
passcode = 'Pw#cbashuju0131'
endpoint = 'cbashuju.ctkaehd5rxxe.us-east-1.rds.amazonaws.com'
database = 'CBA_Data'
engine = create_engine(f'mysql+pymysql://{user_name}:{passcode}@{endpoint}/{database}')
connection= engine.connect()

In [9]:
df.to_sql(name='PlayerStatsPerGame',con=connection,index=False,if_exists='replace')

In [10]:
connection.close()